In [1]:
import warnings # default warn => SettingWithCopyWarning
warnings.simplefilter(action='ignore', category=UserWarning)
import pandas as pd; pd.options.mode.chained_assignment = None
import numpy as np
from datetime import datetime, date
import time
import scipy.fftpack
from scipy.signal import savgol_filter
import matplotlib.pyplot as plt
import arrow
from pprint import pprint as pp
#from tqdm import tqdm

import dmyplant2
from dmyplant2 import (
    cred, MyPlant, Engine,
    FSMOperator, startstopFSM, FSM_splot, FSM_splotBC, FSM_VLine, FSM_add_Notations, FSM_add_Alarms, FSM_add_Warnings,
    bokeh_show, dbokeh_chart, add_dbokeh_vlines, get_cycle_data2, disp_result, disp_alarms, disp_warnings,
    cvset, cplotdef, equal_adjust, count_columns, load_data, get_cycle_data, get_cycle_data2, figures)

import ipywidgets as widgets
from ipywidgets import AppLayout, Button, Layout, VBox, HBox, Label, HTML, interact, interact_manual, interactive, IntSlider, Output
from IPython.display import HTML, display
display(HTML("<style>.container {width:94% !important;}</style>"))

from bokeh.io import push_notebook, show, output_notebook
from bokeh.models import ColumnDataSource, Label, Text, Span, HoverTool #, Range1d#, LabelSet
from bokeh.plotting import figure
output_notebook(hide_banner=True)

# login to myplant()
cred()
mp = MyPlant(0)
Engine._list_cached_validations()
#mp._fetch_installed_base(); # refresh local installed fleet database

,1486152,1330175
Asset ID,159397,114517
Validation Engine,Forsa Hartmoor M02,CREYKE BECK M03
n,999,999
oph@start,54.0,0
serialNumber,1486152,1330175
starts@start,57.0,0
val start,2022-01-10 00:00:00,2017-12-05 00:00:00
source,from_MyPlant,from_MyPlant


In [2]:
def sfun(x):
    return all([ ("CREYKE BECK" in str(x['IB Site Name'])),  (x['OperationalCondition'] != 'Decommissioned') ])
    #return all([ ("Yukon Energy" in str(x['IB Site Name'])),  (x['OperationalCondition'] != 'Decommissioned') ])
    #return all([ ("Forsa Hartmoor" in str(x['IB Site Name'])),  (x['OperationalCondition'] != 'Decommissioned') ])
    #return all([ ("EWB" in str(x['IB Site Name'])),  (x['OperationalCondition'] != 'Decommissioned') ])
    #return all([ ("Pforzheim" in str(x['IB Site Name'])),  (x['OperationalCondition'] != 'Decommissioned') ])
    #return all([ ("BMW Landshut 4.10" in str(x['IB Site Name'])),  (x['OperationalCondition'] != 'Decommissioned') ])
fleet = mp.search_installed_fleet(sfun).drop('index', axis=1)
fleet = fleet.sort_values(by = "Engine ID",ascending=True).reset_index(drop='index')
fleet.T

,0,1,2,3,4,5,6,7,8,9,10
Count_OpHour,7584.0,7592.0,7627.0,7609.0,7601.0,7506.0,7593.0,7596.0,7601.0,7599.0,7597.0
OperationalCondition,Available,Available,Available,Available,Available,Available,Available,Available,Available,Available,Available
shutdown_counter,3840.0,3767.0,3904.0,3851.0,3724.0,3818.0,3713.0,3757.0,3738.0,3779.0,3736.0
startup_counter,3840.0,3767.0,3904.0,3850.0,3725.0,3818.0,3713.0,3756.0,3738.0,3778.0,3735.0
id,114432,114433,114517,114516,114434,114435,114533,114951,115074,115249,115250
Engine Version,H312,H312,H312,H312,H312,H312,H312,H312,H312,H312,H312
Engine Type,624,624,624,624,624,624,624,624,624,624,624
Engine Series,6,6,6,6,6,6,6,6,6,6,6
Country,GB,GB,GB,GB,GB,GB,GB,GB,GB,GB,GB
Commissioning Date,2018-02-05,2017-12-05,2017-12-05,2017-12-05,2017-12-05,2017-12-05,2017-12-05,2017-12-05,2017-12-05,2017-12-05,2017-12-05


In [3]:
motor = fleet.iloc[2]
modes = ['undefined','OFF','MAN','AUTO']
success = ['success','failed','undefined']
e=Engine.from_fleet(mp,motor)
#pp_from='2022-03-28 06:00' # 1 Start
#pp_to='2022-03-28 08:14'
pp_from='2023-03-27'
pp_to='2023-03-28'
motor['IB Site Name'] + ' ' + motor['Engine ID'], modes, success

('CREYKE BECK M03',
 ['undefined', 'OFF', 'MAN', 'AUTO'],
 ['success', 'failed', 'undefined'])

In [4]:
fsm = FSMOperator(e, p_from=pp_from, p_to=pp_to)
fsm.run0(enforce=True, silent=False, debug=False)
fsm.run1(silent=False, successtime=300, debug=False) # run Finite State Machine
fsm.run2(silent=False)
fsm.run4(silent=False)

FSM4: 100%|██████████████████████████████████| 3/3 [00:00<00:00, 19.20 starts/s]


In [5]:
pp_from='2023-03-28'
pp_to='2023-03-29'
fsm2 = FSMOperator(e, p_from=pp_from, p_to=pp_to)
fsm2.run0(enforce=True, silent=False, debug=False)
fsm2.run1(silent=False, successtime=300, debug=False) # run Finite State Machine
fsm2.run2(silent=False)
fsm2.run4(silent=False)

FSM4: 100%|██████████████████████████████████| 3/3 [00:00<00:00, 18.66 starts/s]


In [6]:
print(f"FSM  {fsm.results['starts_counter']} {fsm.results['first_message']} {fsm.results['last_message']}")
print(f"FSM2 {fsm2.results['starts_counter']} {fsm2.results['first_message']} {fsm2.results['last_message']}")


FSM  3 2023-03-27 00:23:24.297999872 2023-03-27 22:23:28.471000064
FSM2 3 2023-03-28 00:23:29.184999936 2023-03-28 23:23:28.510000128


In [7]:
pp_from='2023-03-27'
pp_to='2023-03-29'
fsm3 = FSMOperator(e, p_from=pp_from, p_to=pp_to)
fsm3.run0(enforce=True, silent=False, debug=False)
fsm3.run1(silent=False, successtime=300, debug=False) # run Finite State Machine
fsm3.run2(silent=False)
fsm3.run4(silent=False)
print(f"FSM3 {fsm3.results['starts_counter']} {fsm3.results['first_message']} {fsm3.results['last_message']}")


FSM4: 100%|██████████████████████████████████| 6/6 [00:00<00:00, 17.50 starts/s]
FSM3 6 2023-03-27 00:23:24.297999872 2023-03-28 23:23:28.510000128


In [8]:
for k in fsm.results.keys():
    print(k)

sn
save_date
first_message
last_message
starts
starts_counter
stops
run2_content
run4_content
serviceselectortiming
oilpumptiming
stops_counter
run2_failed
runlog
runlogdetail
run4_failed


In [9]:
print(f"SN: {fsm.results['sn']} - {fsm2.results['sn']}")
print(f"starts: {len(fsm.results['starts'])} - {len(fsm2.results['starts'])}")
print(f"starts: {fsm.results['first_message']} - {fsm2.results['first_message']}")

SN: 1330175 - 1330175
starts: 3 - 3
starts: 2023-03-27 00:23:24.297999872 - 2023-03-28 00:23:29.184999936


In [10]:
fsm.merge_results(fsm2)

ValueError: Merging different Engines 1330175 != 1330175